### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.person-person"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
person_person = df_processados['person_person']

print(f"Colunas: {person_person.shape[1]}\nLinhas: {person_person.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'businessentityid'
duplicadas = person_person[person_person.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['businessentityid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")


#doc*: ID da tabela 

In [ ]:
# Remover duplicadas* 
person_person = person_person.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(person_person)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = person_person.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

In [ ]:
# Ordenar e exibir o df por 'businessentityid'
person_person = person_person.sort_values(by=['businessentityid'])

print(person_person)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in person_person.columns:   
    missing_rows = person_person[person_person[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
#deletar colunas com muitos valores ausentes
person_person = person_person.drop(columns=['title', 'middlename', 'suffix', 'additionalcontactinfo', 'namestyle', 'demographics'])


#doc: As colunas title, middlename, suffix e additionalcontactinfo foram deletadas, pois há muitos valores ausentes.
#     com exceção da coluna namestyle, que há dados, mas como uma coluna booleana, ela só tem false, o que também não agrega valor a análise e
#     da coluna demographics, que também há dados, mas para o momento, não há necessidade de utlizá-la

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in person_person.columns:   
    missing_rows = person_person[person_person[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = person_person.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

In [ ]:
#verificar informações do df
person_person.info()

In [ ]:
# Listar colunas binárias esperadas e verificar valores únicos e suas distribuições
coluna_binaria = ['emailpromotion']

for col in coluna_binaria:
    unique_values = person_person[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")
    print(f"Distribuição de '{col}':")
    print(person_person[col].value_counts())
    print()


#doc: garantir que colunas binárias contenham apenas valores esperados, no caso True ou False e identificar anomalias verificando os valores únicos em cada coluna; valores fora do padrão binário, facilitando a validação e correção


In [ ]:
# Contar o número de pessoas para cada valor de emailpromotion
email_counts = person_person['emailpromotion'].value_counts()

# Estilo do gráfico
sns.set(style='darkgrid', rc={"axes.facecolor": "black", "figure.facecolor": "black"})

plt.figure(figsize=(8, 5))
ax = sns.barplot(
    x=email_counts.index, 
    y=email_counts.values, 
    palette=sns.color_palette("viridis", len(email_counts))
)

plt.title("Quantidade de Emails Promocionais Recebidos", color='white', fontsize=14)
plt.xlabel("Número de Emails Promocionais", color='white', fontsize=12)
plt.ylabel("", fontsize=12)
ax.tick_params(axis='x', colors='white', labelsize=10)
ax.tick_params(axis='y', colors='white', labelsize=10)
ax.set_yticks([])

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', color='white', fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.5, color='gray')
plt.show()


#doc: Com base nos dados: 
#     0 = 11158 pessoas não receberam email promocional
#     1 = 5044 pessoas receberam 1 email promocional e 
#     2 = 3770 pessoas receberam 3 emails promocionais

In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['person_person'] = person_person

In [ ]:
person_person.info()

In [ ]:
# Padronizar colunas com valores textuais
person_person['persontype'] = person_person['persontype'].str.strip().str.upper()
person_person['firstname'] = person_person['firstname'].str.strip().str.upper()
person_person['lastname'] = person_person['lastname'].str.strip().str.upper()
person_person['rowguid'] = person_person['rowguid'].str.strip().str.upper()

print(person_person.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")